In [1]:
import time
import os, psutil
import math
import sys

#Setting Goal State as a Global Variable for ease of access
global goal_state
goal_state = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "0"]

global expanded_nodes
expanded_nodes = 0

#Declaring a Node Class that stores the State, Parent Node, Cost and the Action performed on Parent to reach State
class Node:
    def __init__(self,state,parent,action):
        self.state = state
        self.parent = parent
        self.action = action
        if self.parent is None:
            self.cost = 0
        else:
            self.cost = parent.cost + 1
    def __lt__(self, alternate):
        return (self.state.board < alternate.state.board) and (self.cost < alternate.cost)
    
    def __eq__(self,alternate):
        return self.state.board == alternate.state.board
    
    def __repr__(self):
        return str(self.state.board)

#Declaring Environment and Modification Function that helps identify Child Nodes
class Environment:
    def __init__(self,board):
        self.side = 4 
        self.board = board

    #Performs U, D, L, R Operations on Node to find Children Nodes
    def perform_action(self,action):
        new_board = self.board[:]
        zero_index = new_board.index('0')
        
        if action == "L" and (zero_index % 4)> 0:
            temp_L = new_board[zero_index - 1]
            new_board[zero_index - 1] = new_board[zero_index]
            new_board[zero_index] = temp_L
            
        if action == "R" and (zero_index % 4)< 3:
            temp_R = new_board[zero_index + 1]
            new_board[zero_index + 1] = new_board[zero_index]
            new_board[zero_index] = temp_R
            
        if action == "U" and (zero_index / 4)>= 1:
            temp_U = new_board[zero_index - 4]
            new_board[zero_index - 4] = new_board[zero_index]
            new_board[zero_index] = temp_U
            
        if action == "D" and (zero_index / 4)< 3:
            temp_D = new_board[zero_index + 4]
            new_board[zero_index + 4] = new_board[zero_index]
            new_board[zero_index] = temp_D
        
        return Environment(new_board)

#Manhattan Distance Heuristic Function that takes Node as Input and Outputs Manhattan Distance
def manhattan_distance(node):
    board = node.state.board
    side = 4
    distance = 0
    for i in range(side):
        for j in range(side):
            position = int(board[(i*side)+j])
            if position == 0:
                continue
            distance = distance+abs(i-(position-1)/side)+abs(j-(position-1)%side)
    return distance

#Misplaced Tiles Heuristic Function that takes Node as Input and Outputs Number of Misplaced Tiles
def misplaced_tiles(node):
    board = node.state.board
    side = node.state.side
    length_board = len(board)
    #Initialize Misplaced Tiles as 0
    misplaced_tiles = 0
    for i in range(1, length_board):
        if i == int(board[i-1]):
            continue
        if i != int(board[i-1]):
            misplaced_tiles = misplaced_tiles + 1
    return misplaced_tiles

#Evaluating Heuristic based on User Input
def h(node, heuristic):
    if heuristic == "manhattan_distance":
        h = manhattan_distance(node)
    elif heuristic == "misplaced_tiles":
        h = misplaced_tiles(node)
    return h

#Performs Iterative Deepening with the Heuristic Function (Manhattan or Misplaced Tiles based on User input)
def IDA_star(node, heuristic):
    global expanded_nodes
    path = []
    estimate = h(node, heuristic)
    path.append(node)
    while True:
        t = search(path, 0, estimate, heuristic)
        if t == "GOAL_REACHED":
            return path, estimate
        if t == float("inf"):
            return "NOT GOAL_REACHED"
        estimate = t

def search(path, g, estimate, heuristic):
    global expanded_nodes
    node = path[-1]
    #Evaluation Function
    f = g + h(node, heuristic)
    if f > estimate:
        return f
    if (node.state.board == goal_state):
        return "GOAL_REACHED"
    minimal = float("inf")
    #Finding Children Nodes
    children = []
    actions = ["U", "R", "L", "D"]
    for action in actions:
        child_state = node.state.perform_action(action)
        child_node = Node(child_state, node, action)
        children.append(child_node)
    for successor_node in children:
        #If Node has not been Explored Before
        if successor_node not in path:
            expanded_nodes+=1
            path.append(successor_node)
            t = search(path, g + 1, estimate, heuristic)
            if t == "GOAL_REACHED":
                return "GOAL_REACHED"
            if t < minimal:
                minimal = t
            path.pop()
    return minimal

#Function to find Path Actions from Output of Iterative Deepening Function
def path_finder(path):
    path_taken = []
    for i in range(0, len(path)-1):
        Environment(path[i])
        zero_index_1 = path[i].state.board.index('0')
        zero_index_2 = path[i+1].state.board.index('0')
        difference = zero_index_1 - zero_index_2
        
        if difference == 1:
            move = "L"      
        elif difference == -1:
            move = "R"
        elif difference == 4:
            move = "U"
        elif difference == -4:
            move = "D"
            
        path_taken.append(move)
    return path_taken
    
#Main Function
def main():
    global expanded_nodes
    goal_state = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "0"]
    flag = True
    #Loop to get Inputs from User to Obtain Initial State and Heuristic Choice for Evaluation Function
    while flag:
        print("\n")
        initial_state = str(input("Enter the Initial State of the Puzzle: "))
        initial_state = initial_state.split(" ")
        initial_node = Node(Environment(initial_state), None, None)
        print("--------------------List of Available Operations--------------------")
        print("1. Manhattan Distance")
        print("2. Misplaced Tiles")
        print("3. Exit")
        operation = int(input("Choose Heuristic or Exit the Program: "))
        #Manhattan Heuristic 
        if operation == 1:
            expanded_nodes = 0
            process = psutil.Process(os.getpid())
            initial_memory = process.memory_info().rss / 1024.0
            commencement = time.time()
            path, estimate = IDA_star(initial_node, "manhattan_distance")
            path_taken = path_finder(path)
            print("Moves: ",path_taken)
            print("Number of Expanded nodes: ", expanded_nodes)
            conclusion = time.time()
            final_memory = process.memory_info().rss / 1024.0
            #Used to calculate Memory Usage for IDA Function
            memory_usage = final_memory - initial_memory
            #Used to calculate Time Consumed by IDA Function
            elapsed_time = conclusion - commencement
            print("Time Taken: " +str(round(elapsed_time, 4))+ " seconds")
            print("Memory Used: " +str(memory_usage)+ " kb")
        #Misplaced Tiles Heuristic
        elif operation == 2:
            expanded_nodes = 0
            process = psutil.Process(os.getpid())
            initial_memory = process.memory_info().rss / 1024.0
            commencement = time.time()
            path, estimate = IDA_star(initial_node, "misplaced_tiles")
            path_taken = path_finder(path)
            print("Moves: ",path_taken)
            print("Number of Expanded nodes: ", expanded_nodes)
            conclusion = time.time()
            final_memory = process.memory_info().rss / 1024.0
            #Used to calculate Memory Usage for IDA Function
            memory_usage = final_memory - initial_memory
            #Used to calculate Time Consumed by IDA Function
            elapsed_time = conclusion - commencement
            print("Time Taken: " +str(abs(round(elapsed_time, 4)))+ " seconds")
            print("Memory Used: " +str(abs(memory_usage))+ " kb")
        else:
            flag = False
main()



Enter the Initial State of the Puzzle: 5 2 4 8 10 0 3 14 13 6 11 12 1 15 9 7
--------------------List of Available Operations--------------------
1. Manhattan Distance
2. Misplaced Tiles
3. Exit
Choose Heuristic or Exit the Program: 1
Moves:  ['R', 'D', 'D', 'L', 'L', 'U', 'R', 'D', 'R', 'R', 'U', 'U', 'L', 'L', 'D', 'R', 'R', 'U', 'U', 'L', 'L', 'D', 'L', 'D', 'R', 'R', 'D', 'L', 'U', 'U', 'L', 'U', 'R', 'R', 'D', 'D', 'R', 'D']
Number of Expanded nodes:  16881559
Time Taken: 420.3149 seconds
Memory Used: -392.0 kb


Enter the Initial State of the Puzzle: 5 2 4 8 10 0 3 14 13 6 11 12 1 15 9 7
--------------------List of Available Operations--------------------
1. Manhattan Distance
2. Misplaced Tiles
3. Exit
Choose Heuristic or Exit the Program: 3
